In [393]:
import numpy as np
import json
import os
import pandas as pd

In [394]:
#Toma los archivos .json del directorio y los une a una base de datos usando pandas.

data_list = []
base_dir = 'tech---ops/'

for file in os.listdir(base_dir):
    if 'json' in file:
        json_path = os.path.join(base_dir, file)
        json_data = pd.read_json(json_path, dtype=False)
        data_list.append(json_data)

df = pd.concat(data_list)


In [395]:
# Se agregaron las columnas nuevas segun las reacciones y ademas se ordenaron por el index.

nuevas_columnas_reacciones = ['t_finalizado', 't_revisado', 't_resuelto']

df_columnas_reaccion = df.reindex(columns=df.columns.tolist() + nuevas_columnas_reacciones)
df_columnas_reaccion['reactions'].fillna('', inplace=True)
df_columnas_reaccion = df_columnas_reaccion.reset_index(drop=True)



In [396]:
def merge_dict(lista):
    """Funci'on que une diccionarios en uno solo a travez de listas

    Args:
        lista (list): toma como argumdento una lista no vacia!!! que contenga varios diccionarios
    """
    new_dict = {'name':[], 'users':[]}

    for elemento in lista:
        new_dict['name'].append(elemento['name'])
        new_dict['users'].append(elemento['users'])

    return new_dict

In [397]:
# Se creo una lista vacia para almacenar todos los diccionarios de la columna reactions

lista_dict = []

# Se itero la columna reactions para almacenar los respectivos diccionarios usando la funcion anteriormente creada en lista_dict para despues generar un DataFrame con los datos

for i in df_columnas_reaccion.index:
    if df_columnas_reaccion['reactions'][i] != '':
        lista_dict.append(merge_dict(df_columnas_reaccion['reactions'][i]))
    else:
        lista_dict.append({'name': [], 'users': []})

df_reaction_provisoria = pd.DataFrame(lista_dict)

In [ ]:
# Se creo una lista con los simbolos de reaccion

simbolos_reaccion = ['white_check_mark', 'eyes', 'raised_hands']

# Se itero la base de datos con el fin de almacenar True's & False's en las respectivas columnas de su reaccion

for i in df_reaction_provisoria.index:
    if simbolos_reaccion[0] in df_reaction_provisoria['name'][i]:
        df_columnas_reaccion['t_finalizado'][i] = True

for i in df_reaction_provisoria.index:
    if simbolos_reaccion[1] in df_reaction_provisoria['name'][i]:
        df_columnas_reaccion['t_revisado'][i] = True

for i in df_reaction_provisoria.index:
    if any(simbolos_reaccion[2] in s for s in df_reaction_provisoria['name'][i]):
        df_columnas_reaccion['t_resuelto'][i] = True

df_columnas_reaccion[['t_finalizado', 't_revisado', 't_resuelto']] = df_columnas_reaccion[['t_finalizado', 't_revisado', 't_resuelto']].fillna(value=False)

In [399]:
# Seleccionamos las columnas utiles del df y ademas se elimnaron las filas duplicadas y se ordenaron segun la fecha y hora de la creacion de primer mensaje. 

df_clean = df_columnas_reaccion[['client_msg_id', 'text', 'user', 'ts', 'thread_ts', 'latest_reply', 't_finalizado', 't_revisado', 't_resuelto']].drop_duplicates().sort_values(['ts'])

df_clean = df_clean[df_clean['ts'] >= '1642171431.027300'].reset_index(drop=True)

In [400]:
# Columnas nuevas a agregar

nuevas_columnas = ['nuevo_req', 'ticket_ops', 'ticket_tech', 'urgente']

df_clean_new_columns = df_clean.reindex(columns=df_clean.columns.tolist() + nuevas_columnas)
df_clean_new_columns['latest_reply'].fillna('', inplace=True)

In [ ]:
# Se creo una lista con los simbolos asociado a las nuevas columnas.

simbolos = [':writing_hand:', ':dart:', ':robot_face:',':rotating_light:']

# Se itero el df para asociarle un valor de True or False a las respectivas columnas segun la simbologia del mensaje principal del thread.

for i in df_clean_new_columns.index:
    if df_clean_new_columns['latest_reply'][i] != '':
            for j in range(len(simbolos)):
                if simbolos[j] in df_clean_new_columns['text'][i]:
                    df_clean_new_columns[df_clean_new_columns.columns[9 + j]][i] = True
                else:
                    df_clean_new_columns[df_clean_new_columns.columns[9 + j]][i] = False 

In [ ]:
# Mostrar las primeras 10 filas del DataFrame

df_clean_new_columns.head(10)

In [404]:
# Cambiar el nombre de la columna user a Owner

df_clean_new_columns = df_clean_new_columns.rename(columns = {'user': 'Owner'}, inplace=False)

In [ ]:
df_clean_new_columns.head(10)

In [416]:
# Se creo el DataFrame para los users

data_list = []
base_dir = 'user-data/'

for file in os.listdir(base_dir):
    if 'json' in file:
        json_path = os.path.join(base_dir, file)
        json_data = pd.read_json(json_path, dtype=False)
        data_list.append(json_data)

df_users = pd.concat(data_list)
df_users = df_users[['id', 'name']]

In [417]:
# Cambiar el nombre de la columna id a Owner

df_users = df_users.rename(columns = {'id': 'Owner'}, inplace=False)

In [ ]:
# Combinar el df de user con el df original para obtener los nombres de cada mensaje

data_clean = pd.merge(df_users, df_clean_new_columns, on='Owner', how='outer')

data_clean = data_final[data_final['name'].notna()]
data_clean.sort_values(['ts']).reset_index(drop=True)


In [ ]:
data_clean[data_clean['latest_reply'] != ''].sort_values(['ts']).reset_index(drop=True)